### Reward
1. Agent receive +20 successful dropoff
2. Agent get -10 penalty for illegal pick-up and drop-off locations
3. Sligh negative for not making to the destination after each time-step
    - Agent get -1 point for each step it takes

### Action Space
Size 6. Set of all actions that agent can take in a given state:

South (0), North(1), East(2), West(3), Pickup(4), Dropoff(5)

### States
Four locations (RGYB); pick on passenger at one locaion and drop him off at another. Straight lines are walls. Size 500
- Blue: current passenger pick-up location
- Purple: current destination
- Filled square: taxi
    - Yellow: without passenger
    - Green: with passenger

In [87]:
import gym

# env: unified environment interface
env = gym.make("Taxi-v3",render_mode='ansi').env
env.reset() # reset environment to a new, random state

r = env.render() # renders one frame of env
print(r)
print("Action Space {}".format(env.action_space)) # six actions
print("State Space {}".format(env.observation_space)) # 500 possible states

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


Action Space Discrete(6)
State Space Discrete(500)


In [92]:
# encode a state
# (taxi row, taxi column, passenger index, destination index)
state = env.encode(3,1,2,0) # 
print("State:", state)

# reward table
# {action: [(probability, nextstate, reward, done)]}
# here, probability is always 1.0, meaning if an action is taken, only one possible next state
env.P[328]

State: 328


{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

### Brute Force Approach

In [103]:
env.s = 328
epochs = 0
penalties, reward = 0,0
frames = [] # for animation
done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, truncated, info = env.step(action)

    if reward == -10:
        penalties += 1

    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
    })

    epochs += 1

print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 659
Penalties incurred: 228


In [106]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 659
State: 85
Action: 5
Reward: 20


### Q-Learning
- Agent uses env's rewards to learn the best actions to take in a given state
- Looking at reward for taking an action in the current state, then update Q-value to remember if that action was benefitial
- Values stored in Q-table, map to a (state, action) combination
- Q-value referent "quality" of an action
    - Higher Q-value, higher chance of getting greater rewards
-

In [ ]:
import numpy as np
import random
# initialize Q-table
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# start training
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# for plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100000):
    state = env.reset()[0]
    epochs, penalities, reward, = 0,0,0
    done = False

    while not done:
        if random.uniform(0,1) < epsilon:
            # with epsilon probability, explore action space
            action = env.action_space.sample()
        else:
            # exploit learned values
            action = np.argmax(q_table[state]) # gives index (action) with max value
        
        next_state, reward, done, truncated, info = env.step(action)
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        new_value = (1-alpha)*old_value + alpha*(reward+gamma*next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalities += 1
        
        state = next_state
        epochs += 1
    
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")
            

Episode: 99900
Training finished.



In [172]:
# Q-learning agent performance evaluation
total_epochs, total_penalties = 0,0
episodes = 100

for _ in range(episodes):
    state = env.reset()[0]
    epochs, penalities, reward = 0,0,0

    done = False

    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, truncated, info = env.step(action)

        if reward == -10:
            penalties += 1
        
        epochs += 1
    
    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.96
Average penalties per episode: 0.0


Compared to brute force:

Timesteps taken: 659; Penalties incurred: 228

RL model: Shorter path, less mistakes, higher average rewards per move

### Hyperparameters
$\alpha$: should decrease as knowledge base is larger\
$\gamma$: preference for near term reward should increase (increase gamma) as agent gets closer to the deadline\
$\epsilon$: as trials increase, strategy is more developed, so more exploitation than exploration, $\epsilon$ should decrease over time

Can use GENETIC ALGORITHMS to get the right combination of hyperparameter values

### Q-Table
With large number of states in the environment, Q-table size will grow very large. Need deep neural networks to store information instead.

https://openai.com/index/universe/